# Pruebas Filtro:

- Lenguaje Filtro (Python)
- Crear sistema de web scraping, y crear dataset de al menos 5 categorías , con datos obtenidos a       través del web scraping.
- Las categorías deben de estar numéricamente balanceadas en cantidad.
- Se debe crear un procedimiento automatizado de preprocesamiento .
- Se debe desarrollar un modelo de redes neuronales artificiales con tensor flow/keras , para el       procesamiento de este dataset.
- Se debe crear una api rest , que proporcione por medio de métodos http , un ingreso de datos y una   salida de la clasificación del mismo.

# Clases para periodismo

- Política y economía
- Ciencia, tecnología, salud, cultura, innovación y emprendedorismo
- Viajes, turismo
- Espectáculos, recreación, gastronomía y sociales
- Deportes

**Vamos a hacer que cada línea del dataset sea un párrafo de la web scrapeada. Vamos a obtener una nueva lista de palabras confusas y entrenar el modelo luego de haberlas excluido**

In [ ]:
# La idea es transformar estos programas en un paquete de bibliotecas utilizables por un
# usuario nivel Exccel avanzado. Todos los valores hardcodeados en esta celda serán 
# parámetros configurables de la biblioteca a generar

clases = 5
columna = 3



In [ ]:
from requests import get # la vamos a usar para solicitar la página mediante su URL 
from bs4 import BeautifulSoup # biblioteca para web scraping
import pandas as pd
import string                              
from nltk.corpus import stopwords # para remover palabras que no agregan significado          
from nltk.stem import PorterStemmer # para llevar las palabras a su forma raíz        
from nltk.tokenize import TweetTokenizer # para transformar cada palabra en un token
from IPython.display import clear_output
from collections import defaultdict
import numpy as np
import nltk
nltk.download('stopwords')
import pickle
import random

In [ ]:
def save_obj(obj, name):
       with open(name + '.pkl', 'wb') as f:
           pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
       with open(name + '.pkl', 'rb') as f:
           return pickle.load(f)

In [ ]:
def limpiarTextoCrudo(st):
    res = []
    for t in range(500):
        st = st.replace("[" + str(t) + "]", " ")
    for letra in ["a","b","d","e","f","g","h","i","j"]:
        st = st.replace("[" + letra + "]", " ")
    caracteresARemover = ["(",")","[","]",".",",",'"',"“"]
    for signo in caracteresARemover:
        st = st.replace(signo," ")
    st = st.split()
    hizoJoin = False
    for i in range(len(st) - 1):
        if st[i].isdecimal() and i < len(st) -1 and (
                                   st[i+1].lower() == "ad" or 
                                   st[i+1].lower() == "bc" or 
                                   st[i+1].lower() == "ac" or 
                                   st[i+1].lower() == "dc"):
            res.append("".join([st[i],st[i+1]]))
            hizoJoin = True 
        elif st[i].lower() in {"ad","bc","ac","dc"} and hizoJoin:
            hizoJoin = False    
        elif st[i].lower() not in {"ad","bc","ac","dc"} or not hizoJoin:
            res.append(st[i])
            hizoJoin = False        
    if st[-1].lower() not in {"ad","bc","ac","dc"}: 
        res.append(st[-1])
    return " ".join(res)

In [ ]:
parrafo = "Alexander III of Macedon (Greek: Αλέξανδρος Γʹ ὁ Μακεδών, Aléxandros III ho Makedȏn; 20/21 July 356 BC – 10/11 June 323 BC), commonly known as Alexander the Great (Greek: ὁ Μέγας, ho Mégas), was a king (basileus) of the ancient Greek kingdom of Macedon[a] and a member of the Argead dynasty. He was born in Pella in 356 BC and succeeded his father Philip II to the throne at the age of 20. He spent most of his ruling years on an unprecedented military campaign through western Asia and northeast Africa, and by the age of thirty, he had created one of the largest empires of the ancient world, stretching from Greece to northwestern India.[1][2] He was undefeated in battle and is widely considered one of history's most successful military commanders.[3]\n"
limpiarTextoCrudo(parrafo)

In [ ]:
# Esta clase no la vamos a usar ahora, le mejoraremos en futuras notebooks
class Textos:
    
    def __init__(self, clases, webs):
        self.clases = clases
        self.lista_dataset = []
        self.crear_dataframe(clases, webs)

    def crear_dataframe(self, clases, webs):
        self.lista_dataset.append('TextoCrudo')
        self.lista_dataset.append('TextoLimpio')

        for i in range(1, clases + 1):
            self.lista_dataset.append('Clase' + str(i))

        self.lista_dataset.append('Predicción')
        self.lista_dataset.append('CLASE')
        self.dataframe = pd.DataFrame(columns=self.lista_dataset)
        self.webs = pd.ExcelFile(webs)
        df = self.webs.parse(self.webs.sheet_names[0])
        self.__clases_df = set(df["CÓDIGO"].value_counts().index)

        for i in df.index:
            print(f"Creando Dataframe {i+1} de {len(df.index)}")
            url = df["URL"][i]
            req = get(url)
            statusCode = req.status_code
            html = BeautifulSoup(req.text, "html.parser")

            if statusCode == 200:
                paragraphs = html.select("p")
                cont = 0

                for para in paragraphs:
                    if len(para.text) < 15:
                        continue
                    cont += 1
                    self.dataframe = self.dataframe.append({'TextoCrudo': para.text, 'CLASE': df["CÓDIGO"][i]}, ignore_index=True)

                clear_output(wait=True)

    def limpiar(textoCrudo):
        for i in range(100):
            textoCrudo = textoCrudo.replace("[" + str(i) + "]", "")
        
        for i in ["a","b","c"]:
            textoCrudo = textoCrudo.replace("[" + i + "]", "")
        
        stopwords_english = stopwords.words('english')
        stop        = stopwords.words('english')
        tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
        stemmer = PorterStemmer()    
        texto_tokens = tokenizer.tokenize(textoCrudo)
        sufijos_fecha = ["bc","ac"]

        for i in range(len(texto_tokens.copy())):
            if i > len(texto_tokens) - 1:
                break
            if i !=0:
                if texto_tokens[i] in sufijos_fecha and texto_tokens[i - 1].isnumeric():
                    texto_tokens[i] = texto_tokens[i - 1] + texto_tokens[i]     
                    texto_tokens.pop(i - 1)
                           
        texto_clean  = [ word for word in texto_tokens if word not in stop and word not in string.punctuation]  
        texto_stem   = [stemmer.stem(word) for word in texto_clean] 
        return texto_stem       
      

In [ ]:
lista_dataset = []
lista_dataset.append('TextoCrudo')
lista_dataset.append('TextoLimpio')
lista_dataset.append('TextoProcesado')
for i in range(1, clases + 1):
    lista_dataset.append('Clase' + str(i))
lista_dataset.append('MaxFreq')
lista_dataset.append('Predicción')
lista_dataset.append('CLASE')
dataFrame = pd.DataFrame(columns = lista_dataset)

In [ ]:
textos = dataFrame

In [ ]:
textos

In [ ]:
# Webs.xls es una planilla de conteniendo URLs y la clase a la que corresponde 
# Cada usuario debería crear su propia planilla
# TODO: interfaz para crear la planilla
webs = pd.ExcelFile("Webs.xls")

In [ ]:
df = webs.parse(webs.sheet_names[0])
clases_df = set(df["CÓDIGO"].value_counts().index)
clases_df

In [ ]:
for i in df.index:
    print(f"Creando Dataframe {i+1} de {len(df.index)}")
    url = df["URL"][i]
    req = get(url)
    statusCode = req.status_code
    if statusCode == 200:
        html = BeautifulSoup(req.text, "html.parser")
        paragraphs = html.select("p")
        cont = 0
        for para in paragraphs:
            if len(para.text) < 40:
                continue
            cont += 1
            dataFrame = dataFrame.append({'TextoCrudo': para.text, 'CLASE': df["CÓDIGO"][i]}, ignore_index=True)
    clear_output(wait=True)
dataFrame

In [ ]:
textos = dataFrame
textos.sample(10) 

In [ ]:
textos.to_csv("HistoriaTextosCrudos.csv")

In [ ]:
textos = pd.read_csv("HistoriaTextosCrudos.csv")
textos = textos.iloc[:,1:]
textos.head(10)

In [ ]:
textos["TextoCrudo"][5]

In [ ]:
textos.groupby(by=["CLASE"]).count()

In [ ]:
for i in textos.index:
    clear_output(wait=True)        
    print(f"Limpiando Texto Crudo {i+1} de {len(textos.index)}")
    textos["TextoLimpio"][i] = limpiarTextoCrudo(textos["TextoCrudo"][i])
textos

In [ ]:
i = 0
print()
print()
print(textos["TextoCrudo"][i])
print()
print(textos["TextoLimpio"][i])

In [ ]:
textos.sample(10)

In [ ]:
j = 10
for k in range(j):
    i = int(random.random() * len(textos))
    print()
    print()
    print(textos["TextoCrudo"][i])
    print()
    print(textos["TextoLimpio"][i])

In [ ]:
textos.to_csv("HistoriaTextosLimpio.csv")

In [ ]:
textos = pd.read_csv("HistoriaTextosLimpios.csv")
textos = textos.iloc[:,1:]
textos

In [ ]:
stopwords_english = stopwords.words('english')
stop        = stopwords.words('english')
tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
stemmer = PorterStemmer()    

In [ ]:
for i in textos.index:
    clear_output(wait=True)        
    print(f"Tokenizando Texto Limpio {i+1} de {len(textos.index)}")
    texto_tokens = tokenizer.tokenize(textos["TextoLimpio"][i])
    texto_clean  = [word for word in texto_tokens if word not in stop and word not in string.punctuation]
    texto_stem   = [stemmer.stem(word) for word in texto_clean]
    textos["TextoProcesado"][i] = texto_stem
textos

In [ ]:
j = 10
for k in range(j):
    i = int(random.random() * len(textos))
    print()
    print()
    print("TextoCrudo")
    print(textos["TextoCrudo"][i])
    print()
    print("TextoLimpio")
    print(textos["TextoLimpio"][i])
    print()
    print("TextoProcesado")
    print(textos["TextoProcesado"][i])

In [ ]:
textos.to_csv("HistoriaTextosProcesados.csv")

In [ ]:
textos = pd.read_csv("HistoriaTextosProcesados.csv")
textos = textos.iloc[:,1:]
textos

In [ ]:
textos["TextoProcesado"][0]

In [ ]:
textos["TextoCrudo"][0]

In [ ]:
type(textos["TextoProcesado"][0])

In [ ]:
type(textos["TextoProcesado"][0].split(","))

In [ ]:
textos["TextoProcesado"][0].split(",")

In [ ]:
words = textos["TextoProcesado"][0][1:-1].split(",")
print(words)
print()
print(type(words))
print()
nwords = 0
for word in words:
    print(word, end = " ")
    word = word[1:-1]
    while word[-1:] == "'" or word[-1:] == '"':
          word = limpiarWord(word)           
    while word[0] == "'" or word[0] == '"':
          word = limpiarWordLeft(word)          
    nwords += 1
    print(word)
print()
print("Cantidad de palabras", nwords)

In [ ]:
def limpiarWord(word):
    word = word[:-1]
    return word

def limpiarWordLeft(word):
    word = word[1:]
    return word

def extraeWords(textoProcesado):
    print(type(textoProcesado))
    words = []
    textoProcesado = textoProcesado[1:-1].split(",")
    for word in textoProcesado:
        word = word[1:]
        while word[-1:] == "'" or word[-1:] == '"' or word[-1:] == 'ʹ':
            word = limpiarWord(word)           
        while word[0] == "'" or word[0] == '"' or word[-1:] == 'ʹ':
              word = limpiarWordLeft(word)
        words.append(word)        
    return words    

In [ ]:
words = extraeWords(textos["TextoProcesado"][0])
print(len(words))
print(words)
nwords = 0
for word in words:
    nwords += 1
    print(word)
print(nwords)    

In [ ]:
len(textos["TextoProcesado"][0][1:-1].split(","))

In [ ]:
frecuencias = defaultdict(int)
listaPalabras = []
for i in textos.index:
    for word in textos["TextoProcesado"][i][:-1]:
        while word[-1:] == "'" or word[-1:] == '"':
            word = limpiarWord(word)
        listaPalabras.append(word)    
        frecuencias[(word, textos["CLASE"][i])] = frecuencias.get((word, textos["CLASE"][i]), 0) + 1    
    clear_output( wait = True )
    print(f"Creando Frecuencias {i} de {len(textos.index)}")    
vocabulario = set(listaPalabras)    
frecuencias

In [ ]:
frecuencias = defaultdict(int)
palabras = []
for i in textos.index:
    words = extraeWords(textos["TextoProcesado"][i])
    palabras.extend(words)
    for word in words:  
        frecuencias[(word, textos["CLASE"][i])] = frecuencias.get((word, textos["CLASE"][i]), 0) + 1    
    clear_output( wait = True )
    print(f"Creando Frecuencias {i} de {len(textos.index)}")    
vocabulario = set(palabras)    
frecuencias

In [ ]:
len(vocabulario)

In [ ]:
len(frecuencias)

In [ ]:
"the" in vocabulario

In [ ]:
save_obj(frecuencias, "Frecuencias")
save_obj(vocabulario, "Vocabulario")

In [ ]:
frecuencias = load_obj("Frecuencias")
vocabulario = load_obj("Vocabulario")

In [ ]:
def etiquetas(palabras,frecuencias,clases):
    clases = 5 
    import numpy as np
    x = np.zeros((1, clases + 1))
    for palabra in palabras:
        for t in range(1, clases + 1):
            x[0,t] += frecuencias.get((palabra, t),0)
    return x

In [ ]:
for i in textos.index:
    print(i)
    palabras = []    
    words = extraeWords(textos["TextoProcesado"][i])
    for word in words:
        assert (word in vocabulario),"La palabra " + word + " no está en el vocabulario!"     
        palabras.append(word)    
        for k in range(1, clases + 1):
            textos["Clase" + str(k)][i] = etiquetas(palabras,frecuencias,clases)[0,k]
    clear_output( wait = True )
    print(f"Creando Features {i} de {len(textos.index)}") 
textos

In [ ]:
save_obj(vocabulario, "Vocabulario")
save_obj(frecuencias, "Frecuencias")

In [ ]:
len(vocabulario)

In [ ]:
len(frecuencias)

In [ ]:
textos["TextoCrudo"][0]

In [ ]:
textos["TextoLimpio"][0]

In [ ]:
type(stop)

In [ ]:
"the" in stop

In [ ]:
textos.to_csv("Historia.csv")

In [ ]:
textos = pd.read_csv("Historia.csv")
textos = textos.iloc[:,1:]
textos

In [ ]:
for i in textos.index:
    print(textos.loc[i,"CLASE"] ==  np.argmax(textos.iloc[i,columna:columna + clases].values) + 1)
    textos["MaxFreq"][i] = textos.loc[i,"CLASE"] ==  np.argmax(textos.iloc[i,columna:columna + clases].values) + 1
    clear_output( wait = True )
    print(f"Comparando CLASE y features {i} de {len(textos.index)}") 
textos      

In [ ]:
textos.to_csv("Historia.csv")

In [ ]:
textos = pd.read_csv("Historia.csv")
textos = textos.iloc[:,1:]
textos

In [ ]:
textos.groupby(by=["MaxFreq"]).count()

In [ ]:
rate = 100 * 4857 / (1085 + 4857)
print("Porcentaje de features acertados",rate)

**Vamos a procesar estos datos en una red neuronal**

**Obtuvimos un accuracy del 85% con el modelo neuronal**

In [ ]:
textos = pd.read_csv("Historia.csv")
textos = textos.iloc[:,1:]
textos

In [ ]:
textos.sample(30)

In [ ]:
pifiadas = textos[textos["Predicción"] != textos["CLASE"]]
pifiadas

In [ ]:
pifiadas.groupby(by=["CLASE"]).count()

In [ ]:
cont = 0
cantTrues = 0
for i in pifiadas.index:
    if pifiadas["Predicción"][i]  ==  (np.argmax(pifiadas.iloc[cont,columna:columna + clases]) + 1):
       cantTrues +=1                               
    cont += 1
print("Cantidad de veces que la predicción coincide con la clase con la máxima frecuencia:",cantTrues)
print("Cantidad de total de predicciones pifiadas:",cont)
print("Rate Freq/Pifios:", int(100 * cantTrues/cont)) 

In [ ]:
aciertos = textos[textos["Predicción"] == textos["CLASE"]]
aciertos

In [ ]:
cont = 0
cantTrues = 0
for i in aciertos.index:
    if aciertos["Predicción"][i]  ==  (np.argmax(aciertos.iloc[cont,columna:columna + clases]) + 1):
       cantTrues +=1                               
    cont += 1
print("Cantidad de veces que la predicción coincide con la clase con la máxima frecuencia:",cantTrues)
print("Cantidad de total de predicciones acertadas:",cont)
print("Rate Freq/Aciertos:", int(100 * cantTrues/cont)) 

In [ ]:
pifiadas

In [102]:
textos.sample(10)

,TextoCrudo,TextoLimpio,TextoProcesado,Clase1,Clase2,Clase3,Clase4,Clase5,MaxFreq,Predicción,CLASE
5322,Canadian ethnoarchaeologist Brian Hayden calcu...,Canadian ethnoarchaeologist Brian Hayden calcu...,"['canadian', 'ethnoarchaeologist', 'brian', 'h...",2898.0,2667.0,3197.0,2364.0,5792.0,True,5.0,5
3225,"Nonetheless, Louis has also received praise. T...",Nonetheless Louis has also received praise The...,"['nonetheless', 'loui', 'also', 'receiv', 'pra...",4090.0,5273.0,5592.0,2699.0,3831.0,True,3.0,3
3458,"Philip was confirmed as King of Spain, which r...",Philip was confirmed as King of Spain which re...,"['philip', 'confirm', 'king', 'spain', 'retain...",2676.0,3392.0,4104.0,2681.0,1521.0,True,3.0,3
4941,The Mesopotamian Bronze Age began about 3500 B...,The Mesopotamian Bronze Age began about 3500BC...,"['mesopotamian', 'bronz', 'age', 'began', '350...",1905.0,1969.0,1392.0,1046.0,3838.0,True,5.0,5
2323,"In the Byzantine state, the emperor was the so...",In the Byzantine state the emperor was the sol...,"['byzantin', 'state', 'emperor', 'sole', 'abso...",3424.0,4633.0,2978.0,2825.0,1694.0,True,2.0,2
4065,In 1806 an assembly of Jewish notables was gat...,In 1806 an assembly of Jewish notables was gat...,"['1806', 'assembl', 'jewish', 'notabl', 'gathe...",1806.0,2660.0,2131.0,2740.0,1232.0,True,2.0,4
3329,Immediately tensions rose between Poland and t...,Immediately tensions rose between Poland and t...,"['immedi', 'tension', 'rose', 'poland', 'vario...",1851.0,2126.0,2067.0,2799.0,1392.0,False,4.0,3
1951,Peter appealed to England and Aquitaine's Blac...,Peter appealed to England and Aquitaine's Blac...,"['peter', 'appeal', 'england', ""aquitaine'"", '...",1635.0,2789.0,2367.0,2579.0,567.0,True,2.0,2
5619,Trade networks linked this culture with relate...,Trade networks linked this culture with relate...,"['trade', 'network', 'link', 'cultur', 'relat'...",2812.0,2473.0,1879.0,1404.0,4745.0,True,5.0,5
956,When Babylon fell to the Persian Cyrus the Gre...,When Babylon fell to the Persian Cyrus the Gre...,"['babylon', 'fell', 'persian', 'cyru', 'great'...",5666.0,5259.0,3858.0,2753.0,2603.0,True,1.0,1


In [103]:
# frecuenciasMaximas: diccionario que relaciona cada palabra con la clase
# que tiene la máxima frecuencia de aparición de esa palabra

frecuenciasMaximas = {} 

for word in vocabulario:    
    freqWord = []
    for k in range(clases + 1):
        freqWord.append(0)   
        
    for k in range(1, len(freqWord)):
        freqWord[k] = frecuencias.get((word, k),0)    
    for k in range(1, len(freqWord)):
        frecuenciasMaximas[word] = np.argmax(freqWord)   


print()    
print("Frecuencias máximas")
for key in frecuenciasMaximas.keys():
    print(key,frecuenciasMaximas[key])    

clasesPalabrasFM = {}
for k in range(1,clases + 1):
    clasesPalabrasFM[k] = []     
for word in frecuenciasMaximas.keys():
    clasesPalabrasFM[frecuenciasMaximas[word]].append(word)
    
totV = 0    

# Verificamos que todas las palabras del vocabulario estén en el dicciconario de 
# frecuencias de clases

for k in range(1, clases + 1):
    totV += len(clasesPalabrasFM[k])  
assert(totV == len(vocabulario)),"PALABRAS FALTANTES"    


Frecuencias máximas
mysor 2
niqmaddu 5
goidel 1
rectangl 1
lie 1
aquilin 1
heterodoxi 2
silvia 1
communic 3
length 1
virgin 3
polychor 3
belu 1
iv 2
chauvinist 3
confidant 4
ignobl 2
hippo 1
foreleg 5
salica 2
lebensraum 4
vertigo 1
antioch 2
geta 1
bungl 3
moder 4
photiu 2
sieyè 4
brandenburg 3
hannib 1
fervor 4
pharaoh 1
murder 1
unauthor 3
chambonnièr 3
throwback 3
multi-ethn 2
507bc 1
churchmen 2
casa 3
flamste 3
cm 5
kelli 5
penannular 1
papin 3
pronoun 1
borgo 2
ego 1
tenuous 2
maladapt 5
4500bc 5
preambl 4
1500 3
watermil 2
de-emphas 1
acheiropoieto 2
10th 2
transluc 1
pietrasenta 4
hretha 2
general 4
saskia 2
rioter 1
dodg 4
despit 3
adjoin 1
predict 1
verg 4
baptista 3
canegr 5
claim 3
culinari 1
vysokopetrovski 3
tanit 1
endeavour 2
110 5
bastet 1
raf 4
otho 1
virchow 5
trumpet 1
pike 1
ring 1
1605 3
koran 2
rotto 1
bintanath 1
train 4
fictiti 1
darker 5
ānxī 1
near-rip 5
serv 1
agronomi 1
1172 2
fraser 2
afford 1
591 2
awar 3
1653 3
jacobin 4
scienc 3
volley 3
readili 1
lit

truste 2
1538 3
händel 2
ribbl 2
manumit 4
fiancé 2
thylacoleo 5
jarrig 5
complaint 4
marchfeld 4
crisi 4
so-cal 3
afri 3
129bc 1
coke 3
404 1
colombia 4
ˈræməsiːz 1
withdrew 1
hungari 3
tangibl 2
drawback 5
walid 2
anticip 3
olyokma 3
galen 3
wedge-shap 1
bog 4
settl 2
judaean 1
memelland 4
golfe-juan 4
guerrilla 4
castel 2
unbroken 1
lustrat 4
aquitanian 2
mokriška 5
interpret 5
oligocene-miocen 5
simek 2
front 4
481 2
fabian 1
mongolian 2
friendli 4
salari 2
jason 1
state-legalis 2
e1b1a 5
formerli 2
juror 1
mailer 1
christen 3
wildfir 5
troopship 4
guangxi 4
reproach 1
208 1
minstrel 3
strategist 4
chivalr 2
ben 3
149bc 1
936 2
toubeleki 2
sucr 4
falconieri 3
hing 4
sino-japanes 4
s-shape 5
djoser 1
unquest 4
formio 4
bondon 3
spur 4
salutati 3
riquelm 4
aethiopia 3
mind 3
kitchenwar 2
shilka 3
virginia 3
margarita 3
tri 3
530 1
reintroduct 2
spot 1
sri 5
derrynaflan 1
abassid 2
burli 2
roman-era 1
devic 1
kitchen-midden 5
cinqu 3
undetermin 5
pseudodoxia 3
neutral 4
pieta 1
olden 

draught 2
decemb 4
clung 3
expropri 5
pejor 2
jacq 1
lion-heart 2
couthon 4
pleas 3
irresist 3
over-ag 1
tryptophan 5
elig 4
cato 1
lessini 5
1814 4
rome 1
ʊs 1
tsampouna 2
delport 5
qutuz 2
federalist 4
pre-austronesian 5
office-hold 1
zeman 4
illyria 1
necho 3
discolor 5
foul 3
erigyiu 1
wonambi 5
big-gam 5
albin 4
accus 3
remaind 3
beitass 2
chiefdom 5
reudigni 2
foederati 2
hubert 2
hodder 1
exponenti 5
technician 4
1559 3
dee 3
size 5
985 2
dismember 3
platform 1
patron 1
overseen 3
sive 3
document 1
undoubtedli 3
candi 4
do-noth 2
hellfight 4
iceberg 3
faïenc 5
avid 2
entu 1
lend-leas 4
aisn 4
alban 1
plug 4
north-cotentin 2
camillu 1
eb 3
guibert 4
cist 5
grill 1
maximian 1
alani 2
wander 2
fleet 3
rainulf 2
havr 3
groundwork 4
hemphil 5
heir 3
marshland 1
mondse 5
ethnolinguist 5
bean 5
empathet 2
1303bc 1
islamo-persian 2
deifi 1
unbeliev 2
akropolit 2
clemenceau 4
skopj 2
divrigi 2
human-diseas 5
receiv 3
coin 1
indonesia 5
indus-sarasvati 5
ashrafian 1
1211 2
lotharingia 2
p

dispensation 2
keltoi 1
giraff 3
utica 1
judean 1
muck 2
voivod 3
phrygian 1
grania 2
stanley 3
ww1 4
florentin 3
crato 3
index 5
bite 5
paoli 4
stagnat 4
allow 2
graign 2
lynch 4
mirror 3
affin 1
cardiff 1
neoteni 5
kārni 1
iacta 1
carpent 2
achila 2
unsuit 2
sunlight 1
kinship 5
bloodlet 4
and 5
theretofor 3
lightest 3
epiru 1
uk 4
1046 2
sonar 4
assis 2
swine 2
abélard 2
phraat 1
revenu 3
bellsund 3
assimil 2
vasa 3
inexperi 1
garðar 3
incis 5
almund 2
touchston 5
amminap 1
caraca 4
arl 1
saber-tooth 5
josèph 4
orbitofront 5
auxilia 1
ljubljana 4
sinter 1
ɔ 5
srinivasan 5
adrift 3
meatless 5
badli 4
torr 5
liuzhou 4
sulawesi 5
proto-finn 2
bureau 4
be 5
deadli 1
dismiss 3
wrongli 3
saint-ouen 2
misjudg 3
dismount 4
tune 2
132 1
kissinger 4
ogaden 4
storehous 1
column 1
herculaneum 1
estuari 3
grandfath 3
israelit 1
manchukuo 4
kinsman 3
impati 3
catholicissimu 3
wulfstan 2
warangkula 5
laud 1
clown 2
mouth 3
generos 1
cangamir 3
patterson 5
smith-dorrien 4
northumbria 2
sesklo 5
gar

christina 2
84 2
eve 5
european 3
d'est 3
uddin 2
forest-cov 5
paléolithiqu 5
reihengräb 2
literatur 1
șerban 3
lengthwis 1
sicily 2
leverag 1
schwaben 2
stand-in 2
629 2
mild 5
beevor 4
shephelah 1
lessen 3
sed 1
transylvanian 3
inwyck 3
misspel 2
asid 1
landau 3
hardest 3
counter-insurg 4
deuteronomi 1
mandevil 3
reign 3
argentina 4
holidaymak 4
tiananmen 4
atticu 1
russia 4
abscissa 3
veil 3
belong 5
fizzl 3
utensil 2
foment 4
bamberg 2
trubinella 1
dmanisi 5
anal 1
extend 1
789 2
brussel 3
ashkenaz 1
bassianu 1
discipl 2
breath 5
hyparchi 1
full-grown 3
uncharacterist 4
lexic 1
rákosi 4
trogu 1
louse-born 4
divj 5
wide-rang 1
londinium 1
complexion 4
pelvis 5
gudina 3
franciscan 3
ottmar 2
grey-beard 2
curia 2
deserv 3
customari 2
angeloi 2
bluetooth 2
polyphon 3
ashur-uballit 1
trigger 4
gait 5
poundag 3
inspect 4
concern 4
d'angla 4
tomori 2
drown 4
anshan 5
servil 2
artoi 3
templat 2
pi-ramess 1
forerunn 3
get 3
tissø 5
palaiologoi 2
1125 2
anarchist 4
844 2
shoulder 1
1307 2
re

...

knin 3
curaçao 4
avoid 3
520ad 2
tintagel 2
florida 3
ilha 3
sangoan 5
strikingli 2
limbo 3
sic 3
pre-histor 5
collectiv 4
cristóvão 3
gepida 2
fifteenth-centuri 3
historiæ 2
solí 3
propraetor 1
volcan 5
poll 4
malmaison 4
theobald 2
lection 3
grindal 3
mousterian 5
unifi 2
oceanian 5
epileptoid 1
goré 3
equestrian 1
hesbon 1
johnson 4
mexican 4
ithyphal 5
carew 3
47bc 1
regalia 2
bioarchaeolog 5
garonn 2
166ad 1
crematori 2
left-lean 4
1743 3
many-fold 1
nyungan 5
charax 1
dez 1
ae 5
civilis 5
sultanzad 3
drya 5
disequilibrium 5
táin 1
syriac 3
visibl 5
regiomontanu 3
gunnar 2
decentralis 1
materialist 1
descent 1
limbourg 3
contend 2
describ 1
cartograph 3
callao 4
meroë 1
sullan 1
jar 1
praetorian 1
helmstedt 5
278ad 1
69bc 1
methyl 5
1085 2
unremark 4
aliv 1
1119 2
quran 2
vener 2
barcelo 3
water 1
suarin 2
anti-soci 5
pulp 5
odo 2
seabird 5
khurasan 2
triangular 1
unfamiliar 1
rubicon 1
nationalis 4
guangzhou 3
carolina 3
546 1
thresh 1
oligocen 5
regul 1
baldwin 2
dissatisfi 4
fi

coconut 2
1506 3
pag 3
communard 2
patent 3
leopoldstadt 3
cognomen 1
hume 2
post-utrecht 3
ceccaldi 1
siberian 5
watson 5
electr 3
lepenski 5
persuad 2
sallow 4
vigor 4
natal 3
clemenc 1
mcintyr 3
1234 2
scholastic 2
181 4
lanzarot 2
bulgarian 2
1992 5
margarin 4
eyewit 1
arnhem 5
oursel 2
commendador 3
abu 1
witti 1
kneel 3
lettuc 1
sick 3
lugal-zage-si 1
insignific 1
revis 1
impair 2
parpola 5
buonfiglio 3
brissot 4
lena 3
abram 4
plunder 2
alea 1
commenc 4
erfurt 4
asb 5
loud 2
rudamun 1
attack 4
bernave 4
phu 4
zinc 5
giordani 3
264bc 1
philota 1
speed 1
2009 5
deseado 3
gypsum 1
grenvil 4
andelman 4
bubastit 1
409bc 1
mcmanner 4
vertic 5
κώθων 1
mayuma 1
chandler 4
self-sacrific 3
nguru 5
1663 3
wax 1
northern-latitud 5
escal 4
veto 4
polygyn 5
cornelia 1
pascalin 3
pre-socrat 1
dispos 1
semit 1
1152 2
yasuji 4
unruli 3
ɡa 1
dhghem 5
françoise-athénaï 3
non-buddhist 5
irish-influenc 2
ḥammon 1
arama 1
hunter-kil 4
athlitian 5
dalarna 2
one-year 3
chenab 1
pot-belli 4
brazil 3
hou

garthoff 4
billmen 3
amman 4
seven-point 4
ace 4
syro-arabian 2
brand 2
dogmat 2
douan 3
dispatch 3
larger-brain 5
newest 3
comit 2
non-elect 4
ο 1
pretend 3
block 1
pursu 3
deep-sea 5
lorenzo 3
türkenschanz 3
salt 1
cambys 1
ramessid 1
1547 3
caporetto 4
dvoyevlastiy 4
fatawa-e-alamgiri 2
bonapart 4
cochin 3
pescenniu 1
jew 2
tongu 2
oppos 4
poltavka 5
andagoya 3
souper 4
disempower 5
affluent 1
macedonian 1
acclaim 1
coercion 2
nabulion 4
nairobi 5
489 2
bonfini 3
hmh 4
everyday 2
sung 4
wagay 3
κοπτοπλακοῦς 2
capor 4
dc 4
astronaut 4
ˈræmsiːz 1
imbal 4
utilis 2
peloponnesian 5
jaxart 1
h-ali 4
390 4
ebroin 2
colonna-walewski 4
capitol 1
encourag 3
villain 1
gratifi 2
tanegashima 3
quot 1
unchalleng 3
rozel 5
starvat 4
enclav 2
brute 1
breda 3
balassi 3
feudal 2
aspasioi 1
jobo 5
selleri 3
rearguard 2
orang 3
almighti 3
malik 2
langbarðr 2
reshap 2
£ 3
scorpion 1
séguier 3
croisad 2
eleven 3
clotair 2
davout 4
nobunaga 3
bush 4
siamun 1
demille 1
assiz 2
editor 4
1635 3
promot 4
mont

favor 1
titulatur 3
camomil 5
stun 4
amateur 3
subgroup 2
jalut 2
seek 2
288bc 1
el-ouah 3
1754 3
lubbock 5
langobardorum 2
nationalist 4
aggressor 1
fraxinetum 2
septimiu 1
reattribut 4
zig-zag 5
subsum 2
autocrat 1
espionag 4
hour 4
1928 4
1699 3
sachsen 2
mat 5
state-of-affair 2
shore 2
descriptor 2
quechua 3
program 4
bandit 2
dyrrhachium 1
dinar 2
oriental 4
noli 3
retook 2
niger 1
swabian 2
aberystwyth 1
kippur 4
exacerb 4
1785bc 1
1931 5
lucca 2
draperi 3
leipzig 4
humil 1
boost 4
magan 1
mutual 4
onesicritu 1
tiwi 5
heggarti 5
uncertainti 3
prostrat 1
dissolut 4
reidentifi 5
narmer 1
epipalaeolit 5
auxiliari 1
novaya 3
gustav 1
spillway 1
blind 2
aachen 2
discard 4
ardenn 4
phaulkon 3
ephoru 1
113 1
child-rear 2
354 4
kráľ 2
dean 3
asleep 3
scapegoat 4
vandal 2
debut 4
loyal 4
rhodesiensi 5
luciu 1
ascalon 2
battleworthi 3
farc 2
þing 3
neglig 2
compli 4
overtak 3
lupemban 5
unreason 3
pinch 1
motor 1
1427 3
restructur 4
emit 5
tipoti 5
kilt 1
cúailng 1
underpopul 2
aromat 1
16

hunting-gath 5
cunha 3
chalcidic 1
non-clay 1
sikandernameh 1
zebu 5
para-munda 5
8/ 4
06 4
canterburi 2
semiarid 5
eighteenth-centuri 4
107 5
especi 1
croatian 3
misliya 5
heng 5
girit'in 3
kiln 1
vikingstad 2
rhenish 2
masada 1
nubian 1
intrus 4
lagoon 1
taberna 1
mont-saint-michel 2
brescia 2
invalu 1
intrigu 3
filipino 4
exhaust 3
romano-hellenist 2
asylum 1
censorship 3
round 3
effingham 3
thirty-first 1
dedic 1
1579 3
mithridat 1
keen 3
intermediate-rang 4
xiongnu 1
milovan 4
near-east 5
typhoid 1
myrepso 2
sor 3
loubèr 3
protestant 3
wage 4
justifi 3
isogloss 1
ape 5
œillet 3
accept 3
bagor 5
studia 3
gytha 2
re-rout 4
talker 4
foix 3
compar 5
unpowd 4
banknot 3
land-hold 2
effortlessli 5
salabiyya 2
gener 1
devlin 4
legio 1
three-piec 1
mauger 2
caledonian 1
ingrain 1
alcid 2
sil 1
lohanipur 5
cobo 3
toujour 3
rahman 2
malay 3
machiavellian 3
austro-prussian 4
achaemenid 1
diprotodon 5
keep 3
vicar 2
sahlin 5
julian 1
mandibl 5
1217 2
bóndi 2
780 2
whatsoev 3
1312 2
downriv 1
x

In [104]:
frecuenciasMaxDetalle = {}     
    
for word, value in frecuenciasMaximas.items():
    freqWord = []
    for k in range(clases + 1):
        freqWord.append(0)     
    for k in range(1, len(freqWord)):
        freqWord[k] = frecuencias.get((word, k),0)        
    
    if sum(freqWord) == frecuencias[word, np.argmax(freqWord)]:
        rate = 99
    else:    
        rate = int(100 * frecuencias[word, np.argmax(freqWord)] / sum(freqWord))
    
    frecuenciasMaxDetalle[word] = value, rate - 19
    
print()
print("frecuenciasMaxDetalle")
print()
cont = 0
nconfusas = 0
confusas = []
for key in frecuenciasMaxDetalle.keys():
    cont +=1
    if cont > 100000:
        break
    if frecuenciasMaxDetalle[key][1] < 60:    
        print("PALABRA CONFUSA: ",key, frecuenciasMaxDetalle[key])   
        print(frecuenciasMaximas[key])
        freqWord = []
        for k in range(clases + 1):
            freqWord.append(0)   
        for k in range(1, len(freqWord)):
            freqWord[k] = frecuencias.get((key, k),0)
        
        # print("Mínimo",np.min(freqWord[1:]))
        # print("Suma",sum(freqWord[1:]))
        print()
        rateMinSum =  int(500 *  np.min(freqWord[1:]) / sum(freqWord[1:]))
        if rateMinSum > 25:
            nconfusas += 1
            print(freqWord[1:])
            print("minimo",np.min(freqWord[1:])  )
            print(sum(freqWord[1:]))
            print(freqWord[frecuenciasMaxDetalle[key][0]])
            print("% Mínimo / suma:", int(500 *  np.min(freqWord[1:]) / sum(freqWord[1:]) ) )
            # print("key",type(key),key)
            # print("key.shape",key.split().shape)
            confusas.append(key)
        
print("cantidad de palabras confusas = ",nconfusas)   
len(confusas)
confusas
print("cantidad de palabras confusas = ",nconfusas)   
len(confusas)


frecuenciasMaxDetalle

PALABRA CONFUSA:  mysor (2, 56)
2

PALABRA CONFUSA:  lie (1, 22)
1

[10, 4, 2, 4, 4]
minimo 2
24
10
% Mínimo / suma: 41
PALABRA CONFUSA:  heterodoxi (2, 31)
2

PALABRA CONFUSA:  length (1, 25)
1

PALABRA CONFUSA:  virgin (3, 45)
3

PALABRA CONFUSA:  iv (2, 19)
2

PALABRA CONFUSA:  hippo (1, 47)
1

PALABRA CONFUSA:  antioch (2, 53)
2

PALABRA CONFUSA:  bungl (3, 31)
3

PALABRA CONFUSA:  moder (4, 34)
4

PALABRA CONFUSA:  murder (1, 23)
1

PALABRA CONFUSA:  churchmen (2, 47)
2

PALABRA CONFUSA:  casa (3, 56)
3

PALABRA CONFUSA:  cm (5, 37)
5

PALABRA CONFUSA:  kelli (5, 56)
5

PALABRA CONFUSA:  1500 (3, 27)
3

PALABRA CONFUSA:  10th (2, 59)
2

PALABRA CONFUSA:  general (4, 47)
4

PALABRA CONFUSA:  rioter (1, 31)
1

PALABRA CONFUSA:  despit (3, 11)
3

[32, 36, 60, 54, 15]
minimo 15
197
60
% Mínimo / suma: 38
PALABRA CONFUSA:  adjoin (1, 31)
1

PALABRA CONFUSA:  predict (1, 27)
1

PALABRA CONFUSA:  verg (4, 31)
4

PALABRA CONFUSA:  canegr (5, 56)
5

PALABRA CONFUSA:

minimo 3
38
12
% Mínimo / suma: 39
PALABRA CONFUSA:  warlord (2, 21)
2

PALABRA CONFUSA:  2055bc (1, 31)
1

PALABRA CONFUSA:  william (2, 58)
2

PALABRA CONFUSA:  xi (3, 58)
3

PALABRA CONFUSA:  garrison (3, 18)
3

PALABRA CONFUSA:  bate (2, 41)
2

PALABRA CONFUSA:  inward (4, 47)
4

PALABRA CONFUSA:  wednesday (4, 47)
4

PALABRA CONFUSA:  moistur (3, 31)
3

PALABRA CONFUSA:  choic (3, 29)
3

PALABRA CONFUSA:  conspicu (5, 23)
5

PALABRA CONFUSA:  662 (2, 31)
2

PALABRA CONFUSA:  design (1, 5)
1

[27, 19, 23, 14, 27]
minimo 14
110
27
% Mínimo / suma: 63
PALABRA CONFUSA:  rudimentari (1, 14)
1

PALABRA CONFUSA:  rework (2, 31)
2

PALABRA CONFUSA:  struggl (4, 18)
4

PALABRA CONFUSA:  neither (3, 20)
3

[2, 3, 11, 10, 2]
minimo 2
28
11
% Mínimo / suma: 35
PALABRA CONFUSA:  mm (4, 44)
4

PALABRA CONFUSA:  network (5, 12)
5

[14, 13, 12, 7, 21]
minimo 7
67
21
% Mínimo / suma: 52
PALABRA CONFUSA:  subsequ (5, 5)
5

[27, 26, 22, 17, 30]
minimo 17
122
30
% Mínimo / suma: 69
PALABRA CONFUSA:  

PALABRA CONFUSA:  grandson (3, 21)
3

PALABRA CONFUSA:  uniformli (4, 31)
4

PALABRA CONFUSA:  incurs (2, 37)
2

PALABRA CONFUSA:  1154 (2, 31)
2

PALABRA CONFUSA:  eighth (2, 14)
2

[2, 5, 5, 2, 1]
minimo 1
15
5
% Mínimo / suma: 33
PALABRA CONFUSA:  alberto (1, 31)
1

PALABRA CONFUSA:  feebl (2, 31)
2

PALABRA CONFUSA:  undergon (3, 31)
3

PALABRA CONFUSA:  enghien (4, 47)
4

PALABRA CONFUSA:  modesti (1, 31)
1

PALABRA CONFUSA:  handsom (3, 47)
3

PALABRA CONFUSA:  golden (2, 25)
2

PALABRA CONFUSA:  oppon (4, 11)
4

PALABRA CONFUSA:  junior (1, 31)
1

PALABRA CONFUSA:  wale (2, 23)
2

PALABRA CONFUSA:  decompos (3, 31)
3

PALABRA CONFUSA:  major (4, 9)
4

[67, 75, 72, 109, 66]
minimo 66
389
109
% Mínimo / suma: 84
PALABRA CONFUSA:  unlik (1, 4)
1

[18, 13, 18, 10, 17]
minimo 10
76
18
% Mínimo / suma: 65
PALABRA CONFUSA:  envoy (3, 26)
3

PALABRA CONFUSA:  translat (2, 15)
2

PALABRA CONFUSA:  unchang (1, 17)
1

[4, 1, 3, 2, 1]
minimo 1
11
4
% Mínimo / suma: 45
PALABRA CONFUSA:  adju

PALABRA CONFUSA:  categoris (2, 47)
2

PALABRA CONFUSA:  defus (3, 31)
3

PALABRA CONFUSA:  coat (3, 14)
3

[2, 1, 3, 1, 2]
minimo 1
9
3
% Mínimo / suma: 55
PALABRA CONFUSA:  friedrich (4, 31)
4

PALABRA CONFUSA:  250 (5, 32)
5

PALABRA CONFUSA:  fluent (3, 41)
3

PALABRA CONFUSA:  manioc (3, 31)
3

PALABRA CONFUSA:  forehead (1, 31)
1

PALABRA CONFUSA:  tyrann (1, 21)
1

PALABRA CONFUSA:  join (4, 20)
4

PALABRA CONFUSA:  schedul (4, 31)
4

PALABRA CONFUSA:  one-third (1, 23)
1

PALABRA CONFUSA:  guess (2, 14)
2

PALABRA CONFUSA:  recaptur (2, 31)
2

PALABRA CONFUSA:  rival (1, 19)
1

PALABRA CONFUSA:  enthusiast (2, 31)
2

[2, 6, 1, 2, 1]
minimo 1
12
6
% Mínimo / suma: 41
PALABRA CONFUSA:  guest (1, 31)
1

PALABRA CONFUSA:  ridicul (2, 31)
2

PALABRA CONFUSA:  aurangzeb (2, 31)
2

PALABRA CONFUSA:  autumn (2, 24)
2

PALABRA CONFUSA:  obliter (1, 14)
1

PALABRA CONFUSA:  flavour (2, 14)
2

[1, 2, 1, 1, 1]
minimo 1
6
2
% Mínimo / suma: 83
PALABRA CONFUSA:  ought (1, 14)
1

PALABRA CONF

128
41
% Mínimo / suma: 46
PALABRA CONFUSA:  dissemin (2, 31)
2

PALABRA CONFUSA:  saragossa (2, 31)
2

PALABRA CONFUSA:  truth (1, 31)
1

[7, 2, 3, 1, 1]
minimo 1
14
7
% Mínimo / suma: 35
PALABRA CONFUSA:  hero (1, 21)
1

PALABRA CONFUSA:  birth (3, 21)
3

[7, 16, 29, 8, 11]
minimo 7
71
29
% Mínimo / suma: 49
PALABRA CONFUSA:  remot (5, 14)
5

[2, 4, 2, 2, 5]
minimo 2
15
5
% Mínimo / suma: 66
PALABRA CONFUSA:  stringent (4, 31)
4

PALABRA CONFUSA:  79 (5, 47)
5

PALABRA CONFUSA:  represent (5, 12)
5

[6, 7, 2, 5, 9]
minimo 2
29
9
% Mínimo / suma: 34
PALABRA CONFUSA:  landfal (3, 56)
3

PALABRA CONFUSA:  introduct (5, 17)
5

[4, 8, 8, 8, 16]
minimo 4
44
16
% Mínimo / suma: 45
PALABRA CONFUSA:  parish (4, 47)
4

PALABRA CONFUSA:  hire (3, 16)
3

PALABRA CONFUSA:  substitut (2, 6)
2

[2, 3, 2, 2, 3]
minimo 2
12
3
% Mínimo / suma: 83
PALABRA CONFUSA:  cuzco (4, 47)
4

PALABRA CONFUSA:  expansionist (1, 31)
1

PALABRA CONFUSA:  genoa (3, 49)
3

PALABRA CONFUSA:  theoret (3, 39)
3

[3, 1, 1

...

PALABRA CONFUSA:  bethencourt (2, 41)
2

PALABRA CONFUSA:  reawaken (2, 14)
2

PALABRA CONFUSA:  faeri (2, 31)
2

PALABRA CONFUSA:  roi (3, 14)
3

PALABRA CONFUSA:  reassign (4, 31)
4

PALABRA CONFUSA:  inform (1, 11)
1

[23, 9, 22, 12, 9]
minimo 9
75
23
% Mínimo / suma: 60
PALABRA CONFUSA:  fallout (2, 31)
2

PALABRA CONFUSA:  remarri (3, 21)
3

PALABRA CONFUSA:  consul (1, 38)
1

PALABRA CONFUSA:  resum (3, 14)
3

PALABRA CONFUSA:  carnelian (1, 43)
1

PALABRA CONFUSA:  dawn (5, 43)
5

PALABRA CONFUSA:  amman (4, 31)
4

PALABRA CONFUSA:  dispatch (3, 33)
3

PALABRA CONFUSA:  block (1, 32)
1

[16, 4, 2, 6, 3]
minimo 2
31
16
% Mínimo / suma: 32
PALABRA CONFUSA:  pursu (3, 12)
3

[8, 6, 13, 10, 4]
minimo 4
41
13
% Mínimo / suma: 48
PALABRA CONFUSA:  salt (1, 23)
1

[8, 2, 1, 4, 4]
minimo 1
19
8
% Mínimo / suma: 26
PALABRA CONFUSA:  ramessid (1, 47)
1

PALABRA CONFUSA:  jew (2, 18)
2

PALABRA CONFUSA:  tongu (2, 14)
2

[1, 3, 3, 1, 1]
minimo 1
9
3
% Mínimo / suma: 55
PALABRA CONFUSA:  op

PALABRA CONFUSA:  bookkeep (2, 31)
2

PALABRA CONFUSA:  expect (4, 8)
4

[10, 9, 12, 15, 9]
minimo 9
55
15
% Mínimo / suma: 81
PALABRA CONFUSA:  pump (3, 52)
3

PALABRA CONFUSA:  l (5, 22)
5

[1, 1, 2, 3, 5]
minimo 1
12
5
% Mínimo / suma: 41
PALABRA CONFUSA:  tribut (2, 26)
2

PALABRA CONFUSA:  invalid (3, 31)
3

PALABRA CONFUSA:  deriv (2, 15)
2

PALABRA CONFUSA:  orm (5, 47)
5

PALABRA CONFUSA:  south-western (2, 56)
2

PALABRA CONFUSA:  guerr (4, 56)
4

PALABRA CONFUSA:  santiago (3, 31)
3

PALABRA CONFUSA:  popularis (2, 25)
2

PALABRA CONFUSA:  london (3, 18)
3

[12, 13, 25, 13, 4]
minimo 4
67
25
% Mínimo / suma: 29
PALABRA CONFUSA:  26th (4, 47)
4

PALABRA CONFUSA:  2006 (5, 56)
5

PALABRA CONFUSA:  basra (2, 47)
2

PALABRA CONFUSA:  pragmat (3, 26)
3

PALABRA CONFUSA:  abort (4, 56)
4

PALABRA CONFUSA:  root (2, 19)
2

[13, 25, 11, 6, 10]
minimo 6
65
25
% Mínimo / suma: 46
PALABRA CONFUSA:  baetica (1, 31)
1

PALABRA CONFUSA:  emerald (1, 31)
1

PALABRA CONFUSA:  09 (2, 14)
2

P

PALABRA CONFUSA:  two (4, 4)
4

[133, 120, 130, 148, 96]
minimo 96
627
148
% Mínimo / suma: 76
PALABRA CONFUSA:  woodland (5, 47)
5

PALABRA CONFUSA:  people (4, 53)
4

PALABRA CONFUSA:  success (3, 16)
3

[49, 100, 137, 74, 22]
minimo 22
382
137
% Mínimo / suma: 28
PALABRA CONFUSA:  heard (1, 8)
1

[3, 1, 3, 3, 1]
minimo 1
11
3
% Mínimo / suma: 45
PALABRA CONFUSA:  mortal (5, 18)
5

[3, 4, 2, 1, 6]
minimo 1
16
6
% Mínimo / suma: 31
PALABRA CONFUSA:  congress (4, 52)
4

PALABRA CONFUSA:  mean (2, 12)
2

[35, 57, 20, 18, 52]
minimo 18
182
57
% Mínimo / suma: 49
PALABRA CONFUSA:  mid-jun (4, 47)
4

PALABRA CONFUSA:  corrupt (4, 14)
4

PALABRA CONFUSA:  rectifi (3, 47)
3

PALABRA CONFUSA:  1973 (4, 52)
4

PALABRA CONFUSA:  new (3, 8)
3

[151, 156, 246, 241, 115]
minimo 115
909
246
% Mínimo / suma: 63
PALABRA CONFUSA:  larg (5, 10)
5

[104, 111, 118, 88, 173]
minimo 88
594
173
% Mínimo / suma: 74
PALABRA CONFUSA:  wild (5, 49)
5

PALABRA CONFUSA:  firstli (1, 31)
1

PALABRA CONFUSA:  dista

% Mínimo / suma: 62
PALABRA CONFUSA:  ukrain (4, 33)
4

PALABRA CONFUSA:  toler (2, 26)
2

PALABRA CONFUSA:  harm (5, 14)
5

[2, 1, 3, 2, 4]
minimo 1
12
4
% Mínimo / suma: 41
PALABRA CONFUSA:  documentari (2, 43)
2

PALABRA CONFUSA:  316 (5, 47)
5

PALABRA CONFUSA:  iii (2, 25)
2

PALABRA CONFUSA:  comedi (2, 27)
2

PALABRA CONFUSA:  seven (3, 17)
3

[12, 10, 17, 5, 3]
minimo 3
47
17
% Mínimo / suma: 31
PALABRA CONFUSA:  870 (3, 47)
3

PALABRA CONFUSA:  summar (4, 14)
4

PALABRA CONFUSA:  bulwark (2, 31)
2

PALABRA CONFUSA:  phillip (3, 47)
3

PALABRA CONFUSA:  degre (5, 14)
5

PALABRA CONFUSA:  ransom (2, 35)
2

PALABRA CONFUSA:  secondari (2, 7)
2

[3, 4, 2, 3, 3]
minimo 2
15
4
% Mínimo / suma: 66
PALABRA CONFUSA:  eastern (2, 15)
2

[58, 113, 37, 71, 44]
minimo 37
323
113
% Mínimo / suma: 57
PALABRA CONFUSA:  bright (1, 31)
1

PALABRA CONFUSA:  padua (4, 47)
4

PALABRA CONFUSA:  lang (2, 31)
2

PALABRA CONFUSA:  sein (2, 43)
2

PALABRA CONFUSA:  latin (2, 23)
2

PALABRA CONFUSA:  19

1492

In [116]:
len(vocabulario)

26548

In [117]:
len(confusas)

1492

In [107]:
textos["TextoProcesado"][0]

'[\'alexand\', \'iii\', \'macedon\', \'greek\', \'αλέξανδρος\', \'γʹ\', \'ὁ\', \'μακεδών\', \'aléxandro\', \'iii\', \'ho\', \'makedȏn\', \'20/21\', \'juli\', \'356bc\', \'–\', \'10/11\', \'june\', \'323bc\', \'commonli\', \'known\', \'alexand\', \'great\', \'greek\', \'ὁ\', \'μέγας\', \'ho\', \'méga\', \'king\', \'basileu\', \'ancient\', \'greek\', \'kingdom\', \'macedon\', \'member\', \'argead\', \'dynasti\', \'born\', \'pella\', \'356bc\', \'succeed\', \'father\', \'philip\', \'ii\', \'throne\', \'age\', \'20\', \'spent\', \'rule\', \'year\', \'unpreced\', \'militari\', \'campaign\', \'western\', \'asia\', \'northeast\', \'africa\', \'age\', \'thirti\', \'creat\', \'one\', \'largest\', \'empir\', \'ancient\', \'world\', \'stretch\', \'greec\', \'northwestern\', \'india\', \'undef\', \'battl\', \'wide\', \'consid\', \'one\', "history\'", \'success\', \'militari\', \'command\']'

In [108]:
textos

,TextoCrudo,TextoLimpio,TextoProcesado,Clase1,Clase2,Clase3,Clase4,Clase5,MaxFreq,Predicción,CLASE
0,Alexander III of Macedon (Greek: Αλέξανδρος Γʹ...,Alexander III of Macedon Greek: Αλέξανδρος Γʹ ...,"['alexand', 'iii', 'macedon', 'greek', 'αλέξαν...",5765.0,4813.0,4377.0,2796.0,3298.0,True,1.0,1
1,"During his youth, Alexander was tutored by Ari...",During his youth Alexander was tutored by Aris...,"['youth', 'alexand', 'tutor', 'aristotl', 'age...",5726.0,5416.0,4039.0,3072.0,2520.0,True,1.0,1
2,"Alexander endeavoured to reach the ""ends of th...",Alexander endeavoured to reach the ends of the...,"['alexand', 'endeavour', 'reach', 'end', 'worl...",3986.0,3825.0,3753.0,4337.0,2094.0,False,1.0,1
3,Alexander's legacy includes the cultural diffu...,Alexander's legacy includes the cultural diffu...,"[""alexander'"", 'legaci', 'includ', 'cultur', '...",7024.0,6021.0,3813.0,3304.0,4695.0,True,1.0,1
4,"Alexander was born in Pella, the capital of th...",Alexander was born in Pella the capital of the...,"['alexand', 'born', 'pella', 'capit', 'kingdom...",3105.0,2595.0,2874.0,1062.0,1217.0,True,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...
5937,There are nevertheless a number of contemporar...,There are nevertheless a number of contemporar...,"['nevertheless', 'number', 'contemporari', 'hu...",3415.0,3850.0,3387.0,2612.0,4557.0,True,5.0,5
5938,Evidence suggests big-game hunter-gatherers cr...,Evidence suggests big-game hunter-gatherers cr...,"['evid', 'suggest', 'big-gam', 'hunter-gather'...",2355.0,2270.0,2917.0,2085.0,6659.0,True,5.0,5
5939,Hunter-gatherers would eventually flourish all...,Hunter-gatherers would eventually flourish all...,"['hunter-gather', 'would', 'eventu', 'flourish...",2534.0,2625.0,2667.0,2598.0,5156.0,True,5.0,5
5940,The Archaic period in the Americas saw a chang...,The Archaic period in the Americas saw a chang...,"['archaic', 'period', 'america', 'saw', 'chang...",2173.0,2227.0,2068.0,1861.0,4283.0,True,5.0,5


## Para extraer las palabras confusas debemos hacerlo sobre el dataset total, con y sin pifiadas  

In [109]:
conta = 0
for i in textos.index:
    conta += 1
    if conta > 222222:
        break   
    lista = []
    words = extraeWords(textos["TextoProcesado"][i])
    for word in words:
        assert (word in vocabulario),"La palabra " + word + " no está en el vocabulario!"    
        if word not in confusas:
            lista.append(word) 
    clear_output( wait = True )
    print(f"Procesando {i} de {len(textos.index)}") 
    textos["TextoProcesado"][i] = lista
textos

Procesando 5941 de 5942


,TextoCrudo,TextoLimpio,TextoProcesado,Clase1,Clase2,Clase3,Clase4,Clase5,MaxFreq,Predicción,CLASE
0,Alexander III of Macedon (Greek: Αλέξανδρος Γʹ...,Alexander III of Macedon Greek: Αλέξανδρος Γʹ ...,"[alexand, iii, macedon, greek, αλέξανδρος, γ, ...",5765.0,4813.0,4377.0,2796.0,3298.0,True,1.0,1
1,"During his youth, Alexander was tutored by Ari...",During his youth Alexander was tutored by Aris...,"[youth, alexand, tutor, aristotl, age, philip,...",5726.0,5416.0,4039.0,3072.0,2520.0,True,1.0,1
2,"Alexander endeavoured to reach the ""ends of th...",Alexander endeavoured to reach the ends of the...,"[alexand, endeavour, invad, 326bc, win, victor...",3986.0,3825.0,3753.0,4337.0,2094.0,False,1.0,1
3,Alexander's legacy includes the cultural diffu...,Alexander's legacy includes the cultural diffu...,"[alexander, legaci, cultur, diffus, syncret, c...",7024.0,6021.0,3813.0,3304.0,4695.0,True,1.0,1
4,"Alexander was born in Pella, the capital of th...",Alexander was born in Pella the capital of the...,"[alexand, born, pella, capit, macedon, sixth, ...",3105.0,2595.0,2874.0,1062.0,1217.0,True,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...
5937,There are nevertheless a number of contemporar...,There are nevertheless a number of contemporar...,"[hunter-gather, modif, viabil, hunt, 21st, cen...",3415.0,3850.0,3387.0,2612.0,4557.0,True,5.0,5
5938,Evidence suggests big-game hunter-gatherers cr...,Evidence suggests big-game hunter-gatherers cr...,"[evid, suggest, big-gam, hunter-gather, bere, ...",2355.0,2270.0,2917.0,2085.0,6659.0,True,5.0,5
5939,Hunter-gatherers would eventually flourish all...,Hunter-gatherers would eventually flourish all...,"[hunter-gather, america, plain, unit, state, c...",2534.0,2625.0,2667.0,2598.0,5156.0,True,5.0,5
5940,The Archaic period in the Americas saw a chang...,The Archaic period in the Americas saw a chang...,"[archaic, america, environ, warmer, arid, clim...",2173.0,2227.0,2068.0,1861.0,4283.0,True,5.0,5


## Ahora que el texto ha quedado sin las palabras confusas vamos a volver a extraer los features  

In [110]:
for i in textos.index:
    print(i)
    palabras = []    
    # words = extraeWords(textos["TextoProcesado"][i])
    words = textos["TextoProcesado"][i]
    for word in words:
        assert (word in vocabulario),"La palabra " + word + " no está en el vocabulario!"     
        palabras.append(word)    
        for k in range(1, clases + 1):
            textos["Clase" + str(k)][i] = etiquetas(palabras,frecuencias,clases)[0,k]
    clear_output( wait = True )
    print(f"Creando Features {i} de {len(textos.index)}") 
textos

Creando Features 5941 de 5942


,TextoCrudo,TextoLimpio,TextoProcesado,Clase1,Clase2,Clase3,Clase4,Clase5,MaxFreq,Predicción,CLASE
0,Alexander III of Macedon (Greek: Αλέξανδρος Γʹ...,Alexander III of Macedon Greek: Αλέξανδρος Γʹ ...,"[alexand, iii, macedon, greek, αλέξανδρος, γ, ...",3904.0,2871.0,2440.0,1418.0,1050.0,True,1.0,1
1,"During his youth, Alexander was tutored by Ari...",During his youth Alexander was tutored by Aris...,"[youth, alexand, tutor, aristotl, age, philip,...",4303.0,3997.0,2680.0,1963.0,634.0,True,1.0,1
2,"Alexander endeavoured to reach the ""ends of th...",Alexander endeavoured to reach the ends of the...,"[alexand, endeavour, invad, 326bc, win, victor...",1896.0,1808.0,1492.0,2292.0,153.0,False,1.0,1
3,Alexander's legacy includes the cultural diffu...,Alexander's legacy includes the cultural diffu...,"[alexander, legaci, cultur, diffus, syncret, c...",4258.0,3167.0,1439.0,1138.0,1992.0,True,1.0,1
4,"Alexander was born in Pella, the capital of th...",Alexander was born in Pella the capital of the...,"[alexand, born, pella, capit, macedon, sixth, ...",2378.0,1739.0,2078.0,475.0,520.0,True,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...
5937,There are nevertheless a number of contemporar...,There are nevertheless a number of contemporar...,"[hunter-gather, modif, viabil, hunt, 21st, cen...",516.0,823.0,665.0,329.0,1285.0,True,5.0,5
5938,Evidence suggests big-game hunter-gatherers cr...,Evidence suggests big-game hunter-gatherers cr...,"[evid, suggest, big-gam, hunter-gather, bere, ...",270.0,179.0,446.0,315.0,2275.0,True,5.0,5
5939,Hunter-gatherers would eventually flourish all...,Hunter-gatherers would eventually flourish all...,"[hunter-gather, america, plain, unit, state, c...",370.0,413.0,635.0,1017.0,1846.0,True,5.0,5
5940,The Archaic period in the Americas saw a chang...,The Archaic period in the Americas saw a chang...,"[archaic, america, environ, warmer, arid, clim...",192.0,136.0,221.0,149.0,1165.0,True,5.0,5


In [111]:
for i in textos.index:
    print(textos.loc[i,"CLASE"] ==  np.argmax(textos.iloc[i,columna:columna + clases].values) + 1)
    textos["MaxFreq"][i] = textos.loc[i,"CLASE"] ==  np.argmax(textos.iloc[i,columna:columna + clases].values) + 1
    clear_output( wait = True )
    print(f"Comparando CLASE y features {i} de {len(textos.index)}") 
textos      

Comparando CLASE y features 5941 de 5942


,TextoCrudo,TextoLimpio,TextoProcesado,Clase1,Clase2,Clase3,Clase4,Clase5,MaxFreq,Predicción,CLASE
0,Alexander III of Macedon (Greek: Αλέξανδρος Γʹ...,Alexander III of Macedon Greek: Αλέξανδρος Γʹ ...,"[alexand, iii, macedon, greek, αλέξανδρος, γ, ...",3904.0,2871.0,2440.0,1418.0,1050.0,True,1.0,1
1,"During his youth, Alexander was tutored by Ari...",During his youth Alexander was tutored by Aris...,"[youth, alexand, tutor, aristotl, age, philip,...",4303.0,3997.0,2680.0,1963.0,634.0,True,1.0,1
2,"Alexander endeavoured to reach the ""ends of th...",Alexander endeavoured to reach the ends of the...,"[alexand, endeavour, invad, 326bc, win, victor...",1896.0,1808.0,1492.0,2292.0,153.0,False,1.0,1
3,Alexander's legacy includes the cultural diffu...,Alexander's legacy includes the cultural diffu...,"[alexander, legaci, cultur, diffus, syncret, c...",4258.0,3167.0,1439.0,1138.0,1992.0,True,1.0,1
4,"Alexander was born in Pella, the capital of th...",Alexander was born in Pella the capital of the...,"[alexand, born, pella, capit, macedon, sixth, ...",2378.0,1739.0,2078.0,475.0,520.0,True,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...
5937,There are nevertheless a number of contemporar...,There are nevertheless a number of contemporar...,"[hunter-gather, modif, viabil, hunt, 21st, cen...",516.0,823.0,665.0,329.0,1285.0,True,5.0,5
5938,Evidence suggests big-game hunter-gatherers cr...,Evidence suggests big-game hunter-gatherers cr...,"[evid, suggest, big-gam, hunter-gather, bere, ...",270.0,179.0,446.0,315.0,2275.0,True,5.0,5
5939,Hunter-gatherers would eventually flourish all...,Hunter-gatherers would eventually flourish all...,"[hunter-gather, america, plain, unit, state, c...",370.0,413.0,635.0,1017.0,1846.0,True,5.0,5
5940,The Archaic period in the Americas saw a chang...,The Archaic period in the Americas saw a chang...,"[archaic, america, environ, warmer, arid, clim...",192.0,136.0,221.0,149.0,1165.0,True,5.0,5


In [112]:
textos.groupby(by=["MaxFreq"]).count()

,TextoCrudo,TextoLimpio,TextoProcesado,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE
MaxFreq,,,,,,,,,,
False,1005,1005,1005,1005,1005,1005,1005,1005,1005,1005
True,4937,4937,4937,4937,4937,4937,4937,4937,4937,4937


In [113]:
rate = 100 * 4937 / (1005 + 4937)
print("Porcentaje de features acertados",rate)

Porcentaje de features acertados 83.08650286098957


In [114]:
textos.to_csv("Historia.csv")

In [115]:
textos = pd.read_csv("Historia.csv")
textos = textos.iloc[:,1:]
textos

,TextoCrudo,TextoLimpio,TextoProcesado,Clase1,Clase2,Clase3,Clase4,Clase5,MaxFreq,Predicción,CLASE
0,Alexander III of Macedon (Greek: Αλέξανδρος Γʹ...,Alexander III of Macedon Greek: Αλέξανδρος Γʹ ...,"['alexand', 'iii', 'macedon', 'greek', 'αλέξαν...",3904.0,2871.0,2440.0,1418.0,1050.0,True,1.0,1
1,"During his youth, Alexander was tutored by Ari...",During his youth Alexander was tutored by Aris...,"['youth', 'alexand', 'tutor', 'aristotl', 'age...",4303.0,3997.0,2680.0,1963.0,634.0,True,1.0,1
2,"Alexander endeavoured to reach the ""ends of th...",Alexander endeavoured to reach the ends of the...,"['alexand', 'endeavour', 'invad', '326bc', 'wi...",1896.0,1808.0,1492.0,2292.0,153.0,False,1.0,1
3,Alexander's legacy includes the cultural diffu...,Alexander's legacy includes the cultural diffu...,"['alexander', 'legaci', 'cultur', 'diffus', 's...",4258.0,3167.0,1439.0,1138.0,1992.0,True,1.0,1
4,"Alexander was born in Pella, the capital of th...",Alexander was born in Pella the capital of the...,"['alexand', 'born', 'pella', 'capit', 'macedon...",2378.0,1739.0,2078.0,475.0,520.0,True,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...
5937,There are nevertheless a number of contemporar...,There are nevertheless a number of contemporar...,"['hunter-gather', 'modif', 'viabil', 'hunt', '...",516.0,823.0,665.0,329.0,1285.0,True,5.0,5
5938,Evidence suggests big-game hunter-gatherers cr...,Evidence suggests big-game hunter-gatherers cr...,"['evid', 'suggest', 'big-gam', 'hunter-gather'...",270.0,179.0,446.0,315.0,2275.0,True,5.0,5
5939,Hunter-gatherers would eventually flourish all...,Hunter-gatherers would eventually flourish all...,"['hunter-gather', 'america', 'plain', 'unit', ...",370.0,413.0,635.0,1017.0,1846.0,True,5.0,5
5940,The Archaic period in the Americas saw a chang...,The Archaic period in the Americas saw a chang...,"['archaic', 'america', 'environ', 'warmer', 'a...",192.0,136.0,221.0,149.0,1165.0,True,5.0,5


**Vamos al modelo neuronal**

In [118]:
save_obj(vocabulario, "Vocabulario")
save_obj(frecuencias, "Frecuencias")
save_obj(confusas, "Confusas")

**Logramos un accuracy del 85%**